In [189]:
###############################################
###############################################
########## Created by James Patchett ##########
################## 11/03/19 ###################
###############################################
###############################################

In [190]:
## Simple programme that uses the drivers in the The Stack 
## to run an FMR experiment

In [191]:
#Add to the path the location of the drivers
import sys
sys.path.append(r'C:\Data\James\Scripts\The Stack')

#Import the drivers
from Instruments.anritsu import anritsu
from Instruments.gaussmeter import gaussmeter_455 as gaussmeter
from Instruments.kepco import kepco
from Instruments.lockIn import lockIn
from Instruments.motor import motor

In [192]:
##############################################
############### User Inputs ##################
##############################################

In [193]:
folder_address = r'C:\Data\James\Data\SP75A2\FMR\18.03.19\\' #Address to write the data to in CSV format
data_fn = r'results' #File name to write the data to, a new file will be created for each frequency and angle
meta_fn = r'meta' #File name to write the meta-data to

#Short summary of the experiment
experiment_summ = r'Refinement of the experimental procedure'

## Kepco parameters
#Starting, ending current, and the steps between them (A)
curr_start = -9
curr_end = 9
curr_step = 0.15

## MW source parameters
#Start, end, and step frequency (GHz)
freq_start = 3 #Won't go below 2
freq_end = 30
freq_step = 0.5
MW_power = 13 #dBm

## Pulse parameters
#Frequency of the modulating pulse (Hz)
ref_freq = 421

## Motor parameters
#Start, end, and step angles
angle_start = 0
angle_end = 360
angle_step = 0

## Gradient threshold
#Rate of change of signal at which point it will start to sample the magnetic field more finely
gradient_thres = 20e-6 #V/T
fine_curr_step = curr_step/3

#N.b. if any of the step parameters are set to 0
#The programme will simply skip this loop
#N.b. end parameter is not included in the loop (i.e. [start, end))

In [194]:
###############################################
############ End of user inputs ###############
###############################################

In [195]:
## Set up the drivers for all the instruments

#GPIB0 Addresses and instrument ids
MW_source_add = "GPIB0::23::INSTR"
MW_source_id = "MW source"

gaussmeter_add = "GPIB0::14::INSTR"
gaussmeter_id = "Gaussmeter"

kepco_add = "GPIB0::6::INSTR"
kepco_id = 'Kepco'

lock_in_add = "GPIB0::20::INSTR"
lock_in_id = 'Lock-in amplifier'

motor_id = "Motor"

In [196]:
import visa
import matlab.engine

#Set up the resource manager
rm = visa.ResourceManager()

#Check for connectivity by printing the list of connected resources
rm.list_resources()

#Open a matlab engine to connect to the KEPCO
eng = matlab.engine.start_matlab()

#Add the KEPCO and motor functions folder to the engine's path
eng.addpath(r'C:\Data\James\Scripts\The Stack\Instruments\Kepco_func', nargout = 0)
eng.addpath(r'C:\Data\James\Scripts\The Stack\Instruments\Motor_func', nargout = 0)

#Intialise the instruments
MW_source =  anritsu(MW_source_add, rm, MW_source_id)
gssmtr = gaussmeter(gaussmeter_add, rm, gaussmeter_id)
kpco = kepco(kepco_add, eng, kepco_id)
lck_in = lockIn(lock_in_add, rm, lock_in_id)
mtr = motor(eng, motor_id)

Intailising MW source status variables


In [197]:
## Set-up variables
## User can edit them, but the recommended settings are commented in ()

lck_in_ch = 0 #(0); XY, 1; Rtheta
lck_in_tc = 10 #Time-const, 1s (10)
lck_in_fs = 3 #Lock in filter (24 dBm/oct (3))
lck_in_rf_src = 0 #(0) External source

gssmtr_units = 2 #(2) tesla

sleep_time = 10 #Wait (10) seconds between each measurement

In [198]:
## Set up the microwave source
#Set the modulating pulse to the external source
MW_source.set_pulse_sour('EXTernal2')
#Set the pulse modulation on
MW_source.set_pulse_on(1)
#Set the output power
MW_source.set_power(MW_power)


## Set up the lock-in
#Make display 1 display X
lck_in.set_ch_1(lck_in_ch)
#Set the time-const to 1s- N.b. takes around 5 times
#the TC value for the output to settle (i.e. 1.5s)
lck_in.set_time_const(lck_in_tc)
#Set the filter slope ro 24dBm/oct
lck_in.set_filter_slope(lck_in_fs)
#Set the expand to(10x) for the X channel
lck_in.set_expand(1, 1)
#Set to an external source
lck_in.set_rf_src(lck_in_rf_src)
#Set the sensitivity to (16) 500uV/pA
lck_in.set_sens(16)

##Set up the Gaussmeter
#Set the units to Tesla
gssmtr.set_units(gssmtr_units)
#Set auto-range on
gssmtr.set_auto(1)

##Set up the kepco
#If the first current is negative use a negative voltage
#Otherwise positive
if(curr_start < 0):
    kpco.set_output(0, -20) #0A and 20V
else:
    kpco.set_output(0, 20)

In [199]:
## Write the user inputs to a meta-file
import csv 

#Ensure the freq/angle metadata makes sense
if(angle_step == 0):
    angle_end = angle_start
    
if(freq_step == 0):
    freq_end = freq_start    

with open(folder_address+meta_fn+'.csv', mode = 'w', newline = '') as csv_file:
    #Open the csv reader
    csv_reader = csv.writer(csv_file, delimiter = ',', quotechar = '"',\
                 quoting = csv.QUOTE_MINIMAL)
    
    #Write the experiment summary
    csv_reader.writerow([experiment_summ])
    
    
    #Write the headers
    csv_reader.writerow(['Start current','End current', 'Current step',\
                       'Start frequency', 'End frequency', 'Frequency step', 'Power'\
                       'Reference frequency','Start angle', 'End angle', 'Angle step'])
    
    #Write the meta-data
    csv_reader.writerow([curr_start, curr_end, curr_step, freq_start, freq_end, freq_step,\
                        MW_power, ref_freq, angle_start, angle_end, angle_step])
       

In [200]:
#Increase the angle/frequency step size to 1 to ensure it plays nice 
#with arange
if(angle_step == 0):
    angle_end = angle_start+1
    angle_step = 1
    
if(freq_step == 0):
    freq_step = 1
    freq_end = freq_start+1
    

In [201]:
####################################
####################################
### Start running the experiment ###
####################################
####################################

In [202]:
import time
import numpy as np

#Turn the MW output on
MW_source.set_output_on(1)
MW_source.set_freq(float(freq_start))


#Turn on the KEPCO and start applying current so the auto-offset can
#work on the first run
kpco.set_on(1)
kpco.set_curr(curr_start)

#Move the motor to the starting angle
current_angle = mtr.getPos()

if(current_angle > 180):
    rel_angle = angle_start+360-current_angle
else:
    rel_angle = angle_start-current_angle
    
mtr.move_Rel(float(rel_angle))

## Update the lock-in
time.sleep(30)
lck_in.auto_phase()
lck_in.auto_off(1) #Auto-offset X at the beginning of the measurement
lck_in.auto_reserve() #Probably not that important, but update the dynamic reserve

for angle in np.arange(angle_start, angle_end, angle_step):
    
    print("Angle: "+str(angle))

    for frequency in np.arange(freq_start, freq_end, freq_step):
    
        ## Update the microwave source frequency 
        MW_source.set_freq(float(frequency))
        print("Frequency: "+str(frequency))
        
        #Open the data csv file to write to
        address = folder_address+data_fn+'_'+str(angle)+'deg_'+\
                  str(frequency)+'GHz.csv'

        with open(address, mode = 'w+', newline ='') as data_file:

            data_writer = csv.writer(data_file, delimiter = ',', quotechar = '"',\
                         quoting = csv.QUOTE_MINIMAL)

            #Writer the headers
            data_writer.writerow(['Current \\A', 'Magnetic Field \\T', 'Output voltage \\V',\
                                 'Angle \\deg', 'Frequency \\GHz'])
        data_file.close()

        #Loop over the magentic field values
        for current in np.arange(curr_start, curr_end, curr_step):
                                
            #Change the current
            kpco.set_output(float(current), float(20*np.sign(current)))
            print("Current: "+str(float(current)))
    
            #Wait between measurements
            if(current == curr_start):
                time.sleep(3*sleep_time)
            else:
                time.sleep(sleep_time)

            #List to store the results in
            volt_output = [0, 0, 0]
                
            #Read the output from the lock-in, 3 times and take the average
            for i in range(3):
                volt_output[i] = float(lck_in.read(1))
                
                #Wait (4) seconds so the results are approximately independent
                if(i is not 3):
                    time.sleep(4)
                
            #Calculate the average signal
            ave_output = np.mean(volt_output)
            
            print("Output: "+str(ave_output))
          
            #Read the magnetic field from the gaussmeter
            magnetic_field = gssmtr.read()
            
            #Write the data to the CSV file
            with open(address, mode = 'a', newline ='') as data_file:
                
                data_writer = csv.writer(data_file, delimiter = ',', quotechar = '"',\
                                         quoting = csv.QUOTE_MINIMAL)

                data_writer.writerow([current, magnetic_field, ave_output, angle, frequency])

            data_file.close()
            
            #Estimate the gradient of the voltage signal
            #if it exceeds the threshold value, complete a finer currnet loop
            #Or if the current is sufficently close to zero
            if(current != curr_start and (abs((ave_output-previous_output)/(magnetic_field-previous_field)) > gradient_thres or abs(current) <= 1.5*curr_step)):
                print("Running fine loop")
                
                for fine_current in np.arange(current-curr_step, current, fine_curr_step):
                    
                    kpco.set_output(float(fine_current), float(20*np.sign(fine_current)))
                    print("Current: "+str(float(fine_current)))
    
                    #Wait between measurements
                    time.sleep(sleep_time)

                    #List to store the results in
                    volt_output = [0, 0, 0]
                
                    #Read the output from the lock-in, 3 times and take the average
                    for i in range(3):
                        volt_output[i] = float(lck_in.read(1))
                        
                        #Wait (4) seconds so the results are approximately independent
                        if(i is not 3):
                            time.sleep(4)

                    #Calculate the average signal
                    fine_ave_output = np.mean(volt_output) 
                    
                    #Read the magnetic field from the gaussmeter
                    fine_magnetic_field = gssmtr.read()
                    
                    #Write the data to the CSV file
                    with open(address, mode = 'a', newline ='') as data_file:
                
                        data_writer = csv.writer(data_file, delimiter = ',', quotechar = '"',\
                                      quoting = csv.QUOTE_MINIMAL)

                        data_writer.writerow([fine_current, fine_magnetic_field, fine_ave_output, angle, frequency])

                    data_file.close()

            #Store this loops values in memory    
            previous_output = ave_output
            previous_field = magnetic_field
            
            
    
    ## Update the motor angle
    mtr.move_Rel(float(angle_step))

Angle: 0
Frequency: 3.0
Current: -9.0
Output: 1.86266e-08
Current: -8.85
Output: 2.2351866666666666e-08
Current: -8.7
Output: 2.79398e-08
Current: -8.549999999999999
Output: 2.6077100000000005e-08
Current: -8.399999999999999
Output: 3.228606666666666e-08
Current: -8.249999999999998
Output: 3.539046666666667e-08
Current: -8.099999999999998
Output: 3.663223333333334e-08
Current: -7.9499999999999975
Output: 4.28411e-08
Current: -7.799999999999997
Output: 4.7187266666666674e-08
Current: -7.649999999999997
Output: 4.9049866666666674e-08
Current: -7.4999999999999964
Output: 5.40169e-08
Current: -7.349999999999996
Output: 6.022596666666667e-08
Current: -7.199999999999996
Output: 6.33304e-08
Current: -7.049999999999995
Output: 7.078096666666668e-08
Current: -6.899999999999995
Output: 7.264363333333333e-08
Current: -6.749999999999995
Output: 8.071506666666666e-08
Current: -6.599999999999994
Output: 8.381946666666668e-08
Current: -6.449999999999994
Output: 9.31327e-08
Current: -6.299999999999994

Output: -3.3093099999999997e-06
Current: 5.400000000000034
Output: -3.2981399999999996e-06
Current: 5.5500000000000345
Output: -3.2813766666666665e-06
Current: 5.700000000000035
Output: -3.2689600000000005e-06
Current: 5.850000000000035
Output: -3.2608866666666663e-06
Current: 6.0000000000000355
Output: -3.2484699999999995e-06
Current: 6.150000000000036
Output: -3.245363333333333e-06
Current: 6.300000000000036
Output: -3.2341866666666666e-06
Current: 6.450000000000037
Output: -3.22798e-06
Current: 6.600000000000037
Output: -3.22425e-06
Current: 6.750000000000037
Output: -3.216803333333333e-06
Current: 6.900000000000038
Output: -3.2093533333333335e-06
Current: 7.05000000000004
Output: -3.204383333333333e-06
Current: 7.200000000000038
Output: -3.19942e-06
Current: 7.350000000000037
Output: -3.1932100000000005e-06
Current: 7.500000000000039
Output: -3.1901066666666666e-06
Current: 7.650000000000041
Output: -3.18452e-06
Current: 7.80000000000004
Output: -3.1789299999999997e-06
Current: 7.9

Output: -3.20625e-06
Current: 3.75000000000003
Output: -3.2267400000000002e-06
Current: 3.9000000000000306
Output: -3.2484700000000003e-06
Current: 4.050000000000031
Output: -3.2745433333333334e-06
Current: 4.200000000000031
Output: -3.297516666666667e-06
Current: 4.350000000000032
Output: -3.3204900000000004e-06
Current: 4.500000000000032
Output: -3.3267e-06
Current: 4.650000000000032
Output: -3.30745e-06
Current: 4.800000000000033
Output: -3.25964e-06
Current: 4.950000000000033
Output: -3.2050066666666667e-06
Current: 5.100000000000033
Output: -3.15161e-06
Current: 5.250000000000034
Output: -3.11125e-06
Current: 5.400000000000034
Output: -3.08207e-06
Current: 5.5500000000000345
Output: -3.0640666666666666e-06
Current: 5.700000000000035
Output: -3.0473e-06
Current: 5.850000000000035
Output: -3.0305400000000003e-06
Current: 6.0000000000000355
Output: -3.0187400000000003e-06
Current: 6.150000000000036
Output: -3.01191e-06
Current: 6.300000000000036
Output: -3.00322e-06
Current: 6.450000

Output: -2.84365e-06
Current: 5.400000000000034
Output: -2.81447e-06
Current: 5.5500000000000345
Output: -2.789013333333333e-06
Current: 5.700000000000035
Output: -2.7654200000000004e-06
Current: 5.850000000000035
Output: -2.74928e-06
Current: 6.0000000000000355
Output: -2.7387266666666667e-06
Current: 6.150000000000036
Output: -2.726306666666667e-06
Current: 6.300000000000036
Output: -2.7151300000000005e-06
Current: 6.450000000000037
Output: -2.7083000000000004e-06
Current: 6.600000000000037
Output: -2.70209e-06
Current: 6.750000000000037
Output: -2.69526e-06
Current: 6.900000000000038
Output: -2.6915400000000002e-06
Current: 7.05000000000004
Output: -2.68533e-06
Current: 7.200000000000038
Output: -2.6785e-06
Current: 7.350000000000037
Output: -2.6760166666666666e-06
Current: 7.500000000000039
Output: -2.667943333333333e-06
Current: 7.650000000000041
Output: -2.66608e-06
Current: 7.80000000000004
Output: -2.6636e-06
Current: 7.950000000000038
Output: -2.65925e-06
Current: 8.1000000000

Output: -2.4630533333333335e-06
Current: 8.10000000000004
Output: -2.46057e-06
Current: 8.250000000000043
Output: -2.4574633333333334e-06
Current: 8.400000000000041
Output: -2.45436e-06
Current: 8.55000000000004
Output: -2.4506366666666667e-06
Current: 8.700000000000042
Output: -2.4500166666666665e-06
Current: 8.850000000000044
Output: -2.4493966666666664e-06
Frequency: 5.0
Current: -9.0
Output: -7.53133e-07
Current: -8.85
Output: -7.525123333333334e-07
Current: -8.7
Output: -7.500290000000001e-07
Current: -8.549999999999999
Output: -7.475453333333333e-07
Current: -8.399999999999999
Output: -7.45062e-07
Current: -8.249999999999998
Output: -7.43199e-07
Current: -8.099999999999998
Output: -7.425783333333334e-07
Current: -7.9499999999999975
Output: -7.38232e-07
Current: -7.799999999999997
Output: -7.369903333333333e-07
Current: -7.649999999999997
Output: -7.320233333333334e-07
Current: -7.4999999999999964
Output: -7.270559999999999e-07
Current: -7.349999999999996
Output: -7.26435e-07
Curr

...

Output: -1.5801500000000002e-06
Current: 7.650000000000041
Output: -1.5844999999999998e-06
Current: 7.80000000000004
Output: -1.58512e-06
Current: 7.950000000000038
Output: -1.5801499999999998e-06
Current: 8.10000000000004
Output: -1.574563333333333e-06
Current: 8.250000000000043
Output: -1.5671099999999998e-06
Current: 8.400000000000041
Output: -1.5615233333333331e-06
Current: 8.55000000000004
Output: -1.55532e-06
Current: 8.700000000000042
Output: -1.5509699999999999e-06
Current: 8.850000000000044
Output: -1.546e-06
Frequency: 15.0
Current: -9.0
Output: -1.49633e-06
Current: -8.85
Output: -1.49323e-06
Current: -8.7
Output: -1.49012e-06
Current: -8.549999999999999
Output: -1.4907433333333332e-06
Current: -8.399999999999999
Output: -1.4894999999999999e-06
Current: -8.249999999999998
Output: -1.4876400000000002e-06
Current: -8.099999999999998
Output: -1.48702e-06
Current: -7.9499999999999975
Output: -1.48019e-06
Current: -7.799999999999997
Output: -1.4839166666666668e-06
Current: -7.649

Output: -1.5416566666666667e-06
Current: -7.049999999999995
Output: -1.5466266666666665e-06
Current: -6.899999999999995
Output: -1.5509699999999999e-06
Current: -6.749999999999995
Output: -1.5540733333333333e-06
Current: -6.599999999999994
Output: -1.55594e-06
Current: -6.449999999999994
Output: -1.5578e-06
Current: -6.299999999999994
Output: -1.5602833333333334e-06
Current: -6.149999999999993
Output: -1.55656e-06
Current: -5.999999999999993
Output: -1.5615233333333331e-06
Current: -5.8499999999999925
Output: -1.56525e-06
Current: -5.699999999999992
Output: -1.55966e-06
Current: -5.549999999999992
Output: -1.5615233333333331e-06
Current: -5.3999999999999915
Output: -1.5627666666666665e-06
Current: -5.249999999999991
Output: -1.5627666666666665e-06
Current: -5.099999999999991
Output: -1.5602833333333334e-06
Current: -4.94999999999999
Output: -1.5596633333333332e-06
Current: -4.79999999999999
Output: -1.5596633333333332e-06
Current: -4.64999999999999
Output: -1.5602799999999998e-06
Curre

Output: -1.4329999999999999e-06
Current: -3.599999999999987
Output: -1.4329999999999999e-06
Current: -3.449999999999987
Output: -1.43114e-06
Current: -3.2999999999999865
Output: -1.4323799999999997e-06
Current: -3.149999999999986
Output: -1.43176e-06
Current: -2.999999999999986
Output: -1.43176e-06
Current: -2.8499999999999854
Output: -1.4329999999999999e-06
Current: -2.699999999999985
Output: -1.4367266666666667e-06
Current: -2.5499999999999847
Output: -1.4348633333333334e-06
Current: -2.3999999999999844
Output: -1.43797e-06
Current: -2.249999999999984
Output: -1.43797e-06
Current: -2.0999999999999837
Output: -1.44356e-06
Current: -1.9499999999999833
Output: -1.4448000000000001e-06
Current: -1.799999999999983
Output: -1.44666e-06
Current: -1.6499999999999826
Output: -1.4547333333333332e-06
Current: -1.4999999999999822
Output: -1.4572166666666667e-06
Current: -1.3499999999999819
Output: -1.4572166666666667e-06
Current: -1.1999999999999815
Output: -1.4597e-06
Current: -1.049999999999981

Output: -1.5683566666666666e-06
Current: -0.5999999999999801
Output: -1.567733333333333e-06
Current: -0.44999999999997975
Output: -1.56773e-06
Current: -0.2999999999999794
Output: -1.5708366666666667e-06
Current: -0.14999999999997904
Output: -1.5727e-06
Running fine loop
Current: -0.29999999999997906
Current: -0.24999999999997907
Current: -0.19999999999997908
Current: -0.1499999999999791
Current: 2.1316282072803006e-14
Output: -1.5739433333333333e-06
Running fine loop
Current: -0.14999999999997868
Current: -0.09999999999997869
Current: -0.0499999999999787
Current: 0.15000000000002167
Output: -1.57208e-06
Running fine loop
Current: 2.167710455580618e-14
Current: 0.05000000000002167
Current: 0.10000000000002167
Current: 0.300000000000022
Output: -1.57767e-06
Current: 0.4500000000000224
Output: -1.57953e-06
Current: 0.6000000000000227
Output: -1.5820133333333331e-06
Current: 0.7500000000000231
Output: -1.5832566666666665e-06
Current: 0.9000000000000234
Output: -1.5857400000000002e-06
Curr

KeyboardInterrupt: 

In [203]:
##Reset and switch off all the machines outputting power
kpco.reset()
MW_source.reset()